<a href="https://colab.research.google.com/github/quotation3/TIL/blob/master/NBC_%ED%81%B4%EB%9E%98%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math

class NBC:
    def __init__(self,text, mail_type, words, k =0.5):
        self.text = text
        self.words = words
        self.k = k
        self.mail_type = mail_type
        self.tokens = []
        self.p_spam = []
        self.p_normal = []
        self.log_p_spam = []
        self.log_p_normal = []

        
    def table(self):
        text_split = []
        for i in range(len(self.text)):
            text_split.append(' '.join(self.text[i]).split(' '))
        

        for i in range(len(text_split)):
            self.tokens.extend(text_split[i])
        self.tokens = list(set(self.tokens))

        spam_cnt_list = []
        normal_cnt_list = []
        for i in range(len(self.tokens)):
            spam_cnt = 0
            normal_cnt = 0
            for j in range(len(text_split)):    #tokens의 단어가 text_split[j]에 있으면 그 빈도 만큼 카운트 추가
                if self.tokens[i] in text_split[j]:
                    if self.mail_type[j] == 'spam':
                        spam_cnt+=text_split[j].count(self.tokens[i])
                    elif self.mail_type[j] == 'normal':
                        normal_cnt+=text_split[j].count(self.tokens[i])
                    else:
                        pass
            spam_cnt_list.append(spam_cnt)
            normal_cnt_list.append(normal_cnt)

        for i in range(len(self.tokens)):
            prob_spam = (0.5+float(str(spam_cnt_list[i])))/(2*0.5+sum(spam_cnt_list))*100
            prob_normal = (0.5+float(str(normal_cnt_list[i])))/(2*0.5+sum(normal_cnt_list))*100
            self.p_spam.append("%0.2f" %prob_spam)
            self.p_normal.append("%0.2f" %prob_normal)
            log_prob_spam = np.log((0.5+float(str(spam_cnt_list[i])))/(2*0.5+sum(spam_cnt_list)))
            log_prob_normal = np.log((0.5+float(str(normal_cnt_list[i])))/(2*0.5+sum(normal_cnt_list)))
            self.log_p_spam.append("%0.4f"%log_prob_spam)
            self.log_p_normal.append("%0.4f"%log_prob_normal)

        total_table = np.array([self.tokens,spam_cnt_list,normal_cnt_list,self.p_spam,self.p_normal,self.log_p_spam,self.log_p_normal])
        return pd.DataFrame(total_table.T, columns=['단어','spam','normal','P(w|spam)','P(w|normal)','Log(P(w|spam))','Log(P(w|normal))'])

    def spam_filtering (self):
        index_a = self.tokens.index(self.words[0])
        index_b = self.tokens.index(self.words[1])

        normal_prob = np.log(self.mail_type.count('normal')/len(self.mail_type))
        spam_prob = np.log(self.mail_type.count('spam')/len(self.mail_type))
        x1 = math.exp(float(self.log_p_normal[index_a])+(float(self.log_p_normal[index_b]))+(float("%0.4f"%normal_prob)))
        x2 = math.exp(float(self.log_p_spam[index_a])+(float(self.log_p_spam[index_b]))+(float("%0.4f"%spam_prob)))

        return ('{},{}가 포함된 메일이 스팸일 확률 : {}'.format(self.words[0],self.words[1],(x2/(x1+x2))*100), '{},{}가 포함된 메일이 정상일 확률 : {}'.format(self.words[0],self.words[1],(x1/(x1+x2))*100))

In [2]:
text = [['me free lottery'],['free get free you'],['you free scholarship'],['free to contact me'],['you won award'],['you ticket lottery']]
mail_type = ['spam','spam','normal','normal','normal','spam']
words = ['free','lottery']
nbc = NBC(text,mail_type,words)

In [3]:
nbc.table()

,단어,spam,normal,P(w|spam),P(w|normal),Log(P(w|spam)),Log(P(w|normal))
0,me,1,1,13.64,13.64,-1.9924,-1.9924
1,award,0,1,4.55,13.64,-3.0910,-1.9924
2,get,1,0,13.64,4.55,-1.9924,-3.0910
3,to,0,1,4.55,13.64,-3.0910,-1.9924
4,you,2,2,22.73,22.73,-1.4816,-1.4816
5,contact,0,1,4.55,13.64,-3.0910,-1.9924
6,won,0,1,4.55,13.64,-3.0910,-1.9924
7,scholarship,0,1,4.55,13.64,-3.0910,-1.9924
8,lottery,2,0,22.73,4.55,-1.4816,-3.0910
9,free,3,2,31.82,22.73,-1.1451,-1.4816


In [4]:
nbc.spam_filtering()

('free,lottery가 포함된 메일이 스팸일 확률 : 87.49988899428504',
 'free,lottery가 포함된 메일이 정상일 확률 : 12.500111005714952')